In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"



In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from transformers import pipeline
import torch
import os


In [ ]:
# Load dataset
df = pd.read_excel("./TaylorSwiftSongs.xlsx")
df = df.dropna(subset=["LYRICS"])
df = df[df["LYRICS"].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0)]
lyrics_texts = df["LYRICS"].tolist()

# Create HuggingFace Dataset
dataset = Dataset.from_dict({"text": lyrics_texts})

In [ ]:
# Load GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT2 doesn't have pad token by default
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
# Tokenization
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-taylorswift",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=50,
    save_steps=200,
    save_total_limit=1,
    prediction_loss_only=True,
    fp16=torch.cuda.is_available()
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train model
trainer.train()

# Save model
trainer.save_model("./gpt2-taylorswift")
tokenizer.save_pretrained("./gpt2-taylorswift")


<ipython-input-19-40ecffefb030>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.204900
100,2.902900
150,2.736900


('./gpt2-taylorswift/tokenizer_config.json',
 './gpt2-taylorswift/special_tokens_map.json',
 './gpt2-taylorswift/vocab.json',
 './gpt2-taylorswift/merges.txt',
 './gpt2-taylorswift/added_tokens.json')

In [ ]:
# Sample predictions
generator = pipeline('text-generation', model='./gpt2-taylorswift', tokenizer=tokenizer)


prompts = [
    "We were both young when I first saw you",
    "I remember tears streaming down your face",
    "Loving him is like driving a new Maserati",
    "we were both young when I first saw you",
]

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    output = generator(prompt, max_length=100, num_return_sequences=1)[0]["generated_text"]
    print(f"Generated Lyrics:\n{output}")

Device set to use cpu



Prompt: We were both young when I first saw you
Generated Lyrics:
We were both young when I first saw you
You were my beautiful, blue eyed boyfriend, who lived just minutes away from me
You were my only love, a small family with me, who would kiss, drink
Baby, come back again if you do
Baby, if you do it ever again
Baby, if you do it ever again
Baby-even if you don't, you'd take it back if nobody knew

When I didn't find you, my old secret

Prompt: I remember tears streaming down your face
Generated Lyrics:
I remember tears streaming down your face as you gave me the lead over the cliff
I remember feeling the first blood when you put the gun in my face
I remember your laugh and how you said sorry
And 'cause we've had that moment

We just got a little under our skin
It was a pretty sight
Takeaway, I got my first girlfriend tonight
We danced tonight on a rooftop
That kind of makes me wonder if I can be a better girlfriend than you


Prompt: Loving him is like driving a new Maserati
Gene

In [ ]:
generator = pipeline('text-generation', model='./gpt2-taylorswift', tokenizer=tokenizer)


prompts = [
    "Nice to meet you,where you been",
    "I'm pretty sure we broke up last night",
]

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    output = generator(prompt, max_length=100, num_return_sequences=1)[0]["generated_text"]
    print(f"Generated Lyrics:\n{output}")

Device set to use cpu



Prompt: Nice to meet you,where you been
Generated Lyrics:
Nice to meet you,where you been all day and how I feel?
I know I feel better now.
But I can't stop thinking about you now
Because it's been a long time since I've met you
I can see you in my past
I've been a pretty easy girl when I'm dating you,
But don't see me looking ahead anymore

[Chorus]
Oh, oh, oh
I can see you, you can see my eyes

Prompt: I'm pretty sure we broke up last night
Generated Lyrics:
I'm pretty sure we broke up last night."
And then the doorbell rings and the rest is history
In that picture there's something on your phone that's a big deal
But this time it's a little more strange, so I try not to
I've stopped thinking about it
And to think that this is the end
[Chorus]
It's pretty clear that nothing's changed
Your boyfriend's not my boyfriend anymore
She's my girlfriend (the last time
